In [14]:
import warnings

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset

from utils import preprocess_data, bayesian_optimisation

In [2]:
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test, train_df, test_df = preprocess_data(standardise=True)

Customise the data to be fed in terms of tensors

In [3]:

class LoanDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = LoanDataset(X_train, y_train)
test_dataset = LoanDataset(X_test, y_test)

In [4]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Customise a LeNet5 that was used to analyse image data to be used in this project 

In [17]:
class TabularNet(nn.Module):
    def __init__(self, input_size, hidden_layers, dropout_rate, name=None):
        super(TabularNet, self).__init__()
        if name:
            self.name = name
        self.hidden_layers = nn.ModuleList()
        last_size = input_size
        for hidden_layer_size in hidden_layers:
            self.hidden_layers.append(nn.Linear(last_size, hidden_layer_size))
            self.hidden_layers.append(nn.Dropout(dropout_rate))
            last_size = hidden_layer_size
        self.output_layer = nn.Linear(last_size, 2)  # Output layer for binary classification

    def forward(self, x):
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        x = self.output_layer(x)
        return x

The following function trains and evaluates the NN with specified hyperparameters and returns its accuracy



In [18]:
def train_and_evaluate(hidden_layer_sizes, learning_rate, dropout_rate, num_epochs=25):
    model = TabularNet(input_size=X_train.shape[1], hidden_layers=hidden_layer_sizes, dropout_rate=dropout_rate)
    # Loss funcrion optimiser
    criterion = nn.CrossEntropyLoss()  # Classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Evaluation
    model.eval()  # Set the model in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # resource efficiency purpose
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy


Obtain the hyperparameters for the best performing network

In [7]:
def optimize_tabular_net(hidden_layer1, hidden_layer2, learning_rate, dropout_rate):
    hidden_layers = [int(hidden_layer1), int(hidden_layer2)]
    return train_and_evaluate(hidden_layers, learning_rate, dropout_rate)

param_space = np.array([
    (10, 100),  # hidden_layer1 size
    (10, 100),  # hidden_layer2 size
    (0.0001, 0.01),  # learning_rate
    (0.1, 0.5)   # dropout_rate
])

n_iters = 25
initial_samples = 5

# Initial random samples
x0 = np.random.uniform(param_space[:, 0], param_space[:, 1], size=(initial_samples, param_space.shape[0]))
y0 = np.array([optimize_tabular_net(*params) for params in x0])

gp_params = {"alpha": 1e-6}

X_sample, Y_sample, gpr = bayesian_optimisation(n_iters, optimize_tabular_net, param_space, x0, y0.reshape(-1, 1), gp_params)

# Best parameters
best_idx = np.argmax(Y_sample)
best_params = X_sample[best_idx]
best_accuracy = Y_sample[best_idx]

print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")


Best parameters: [7.48656891e+01 1.00000000e+02 1.00000000e-04 1.00000000e-01]
Best accuracy: [0.79674797]


Train the network with the best hyperparameters to be used in predictions later

In [8]:
# Use the best hyperparameters found from Bayesian Optimization
best_hidden_layer1 = int(best_params[0])
best_hidden_layer2 = int(best_params[1])
best_learning_rate = best_params[2]
best_dropout_rate = best_params[3]

best_hidden_layers = [best_hidden_layer1, best_hidden_layer2]

final_model = TabularNet(input_size=X_train.shape[1], hidden_layers=best_hidden_layers, dropout_rate=best_dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_model.parameters(), lr=best_learning_rate)

# Training the final model
num_epochs = 25
final_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = final_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

print("Final model trained.")


Final model trained.


In [9]:
final_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = final_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Final model accuracy: {accuracy:.4f}")


Final model accuracy: 0.7886


Produce a validation dataset based on the network to be used by the other models in the project

In [11]:
X_test_final = test_df.drop(columns=['Loan_ID'])

scaler = StandardScaler()
X_test_final_transformed = scaler.fit_transform(X_test_final)
X_new_tensor = torch.tensor(X_test_final_transformed, dtype=torch.float32)

final_model.eval()
with torch.no_grad():
    outputs = final_model(X_new_tensor)
    _, predicted = torch.max(outputs.data, 1)
    predictions = predicted.numpy()

target_filename = "data/loan_sanction_test_with_predictions_lenet5.csv"
test_df['Loan_Status'] = predictions

test_df.to_csv(target_filename, index=False)

print(f"Predictions have been saved to {target_filename}.")

Predictions have been saved to data/loan_sanction_test_with_predictions_lenet5.csv.


Cross validate this NN by means of the validation set produced by the other models' precitions

## It can be seen that the models performs well with most of the data produced by the other models and exceptionally well against the validation data constructed usign LR

In [19]:
for algo, filename in {
    "CNN" : 'data/loan_sanction_test_with_predictions_cnn.csv',
    "DT": 'data/loan_sanction_test_with_predictions_decision_tree.csv',
    "KNN": 'data/loan_sanction_test_with_predictions_knn.csv',
    "LR": 'data/loan_sanction_test_with_predictions_lr.csv',

}.items():
    test_df_new = pd.read_csv(filename)
    X_new = test_df_new.drop(columns=['Loan_ID', 'Loan_Status'])
    y_new = test_df_new['Loan_Status']
    # Only scale for those that were scaled
    scaler = StandardScaler()
    X_new = scaler.fit_transform(X_new)

    with torch.no_grad():
        outputs = final_model(X_new_tensor)
        _, predicted = torch.max(outputs.data, 1)
        y_pred = predicted.numpy()

    y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to class labels

    lr_accuracy = accuracy_score(y_new, y_pred)
    lr_report = classification_report(y_new, y_pred)
    print(f'LeNet5 Performance for {algo} produced predictions {lr_accuracy}')

LeNet5 Performance for CNN produced predictions 0.16076294277929154
LeNet5 Performance for DT produced predictions 0.9482288828337875
LeNet5 Performance for KNN produced predictions 0.4713896457765668
LeNet5 Performance for LR produced predictions 1.0
